# Import Dependencies and CSV Files

In [1]:
import pandas as pd
import numpy as np
import re
import time

from pathlib import Path
from collections import Counter

In [2]:
# File Paths
credits = "Data/credits.csv"
links = "Data/links.csv"
movies = "Data/movies.csv"
metadata = "Data/movies_metadata.csv"
ratings = "Data/ratings.csv"

# Read CSV Files
credit_df = pd.read_csv(credits, low_memory=False)
link_df = pd.read_csv(links, low_memory=False)
movie_df = pd.read_csv(movies, low_memory=False)
meta_df = pd.read_csv(metadata, low_memory=False)
ratings_df = pd.read_csv(ratings, low_memory=False)

In [3]:
movie_df.tail(20)

,movieId,title,genres
58078,193845,Dywizjon 303 (2018),War
58079,193849,Rafinha Bastos: Ultimatum (2018),(no genres listed)
58080,193851,U: July 22 (2018),Drama|Thriller
58081,193853,Wu Kong (2017),Action|Adventure|Fantasy
58082,193855,"Spring Night, Summer Night (1967)",(no genres listed)
58083,193857,Beautiful Women (2005),Drama
58084,193859,"You, Me and Him (2018)",Comedy
58085,193861,Cold Harvest (1998),Action|Sci-Fi|Thriller
58086,193863,Cocaine Godmother (2017),Documentary|Drama
58087,193864,No somos de piedra (1968),Comedy


# Split Movie Title and Year, Create Separate Columns

In [4]:
# pattern = '\((\d{4})\)'
# years = movie_df["title"].str.extract(pattern, expand=False)

pattern_2 = '(.*\w*)(?:\s\((\d{4})\))'
title = movie_df["title"].str.extract(pattern_2, expand=False)
title["Title"] = title[0]
title["Year"] = title[1]
movie_df["Title"] =  title["Title"]
movie_df["Year"] = title["Year"]
movie_df.head(20)

,movieId,title,genres,Title,Year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995
5,6,Heat (1995),Action|Crime|Thriller,Heat,1995
6,7,Sabrina (1995),Comedy|Romance,Sabrina,1995
7,8,Tom and Huck (1995),Adventure|Children,Tom and Huck,1995
8,9,Sudden Death (1995),Action,Sudden Death,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,GoldenEye,1995


# Separate Genres Into Lists

In [5]:
movie_df.reset_index

movies_df = movie_df.set_index("movieId")
movies_df["genres"] = movies_df["genres"].str.split('|', expand=False)
movies_df["genres"] = movies_df["genres"].tolist()
movies_df = movies_df.drop(columns="title")

# movies_df = movies_df.loc[:, ['Title', 'Year', 'genres']]

# Constructing Function To Split Genre List Into Separate Columns With Boolean Entries

In [6]:
# def genre_split(column):
#     for entry in column:
#         if "Adventure" in entry:
#             movies_df["Adventure"] = 1
#         else: 
#             movies_df["Adventure"] = 0
#     for entry in column:
#         if "Action" in entry:
#             movies_df["Action"] = 1
#         else: 
#             movies_df["Action"] = 0
#     for entry in column:
#         if "Animation" in entry:
#             movies_df["Animation"] = 1
#         else: 
#             movies_df["Animation"] = 0
#     for entry in column:
#         if "Children" in entry:
#             movies_df["Children"] = 1
#         else: 
#             movies_df["Children"] = 0
#     for entry in column:
#         if "Comedy" in entry:
#             movies_df["Comedy"] = 1
#         else: 
#             movies_df["Comedy"] = 0
#     for entry in column:
#         if "Crime" in entry:
#             movies_df["Crime"] = 1
#         else: 
#             movies_df["Crime"] = 0
#     for entry in column:
#         if "Documentary" in entry:
#             movies_df["Documentary"] = 1
#         else: 
#             movies_df["Documentary"] = 0
#     for entry in column:
#         if "Drama" in entry:
#             movies_df["Drama"] = 1
#         else: 
#             movies_df["Drama"] = 0
#     for entry in column:
#         if "Fantasy" in entry:
#             movies_df["Fantasy"] = 1
#         else: 
#             movies_df["Fantasy"] = 0
#     for entry in column:
#         if "Film-Noir" in entry:
#             movies_df["Film-Noir"] = 1
#         else: 
#             movies_df["Film-Noir"] = 0
#     for entry in column:
#         if "Horror" in entry:
#             movies_df["Horror"] = 1
#         else: 
#             movies_df["Horror"] = 0
#     for entry in column:
#         if "IMAX" in entry:
#             movies_df["IMAX"] = 1
#         else: 
#             movies_df["IMAX"] = 0
#     for entry in column:
#         if "Musical" in entry:
#             movies_df["Musical"] = 1
#         else: 
#             movies_df["Musical"] = 0
#     for entry in column:
#         if "Mystery" in entry:
#             movies_df["Mystery"] = 1
#         else: 
#             movies_df["Mystery"] = 0
#     for entry in column:
#         if "Romance" in entry:
#             movies_df["Romance"] = 1
#         else: 
#             movies_df["Romance"] = 0
#     for entry in column:
#         if "Sci-Fi" in entry:
#             movies_df["Sci-Fi"] = 1
#         else: 
#             movies_df["Sci-Fi"] = 0
#     for entry in column:
#         if "Thriller" in entry:
#             movies_df["Thriller"] = 1
#         else: 
#             movies_df["Thriller"] = 0
#     for entry in column:
#         if "War" in entry:
#             movies_df["War"] = 1
#         else: 
#             movies_df["War"] = 0
#     for entry in column:
#         if "Western" in entry:
#             movies_df["Western"] = 1
#         else: 
#             movies_df["Western"] = 0
#     for entry in column:
#         if "None" in entry:
#             movies_df["No Listed Genre"] = 1
#         else: 
#             movies_df["No Listed Genre"] = 0
#     for entry in column:
#         if "(no genres listed)" in entry:
#             movie_df["No Listed Genre"] = 1
#         else:
#             movies_df["No Listed Genre"] = 0
#     return movie_df.head(10)    

In [ ]:
def genre_split(column):
    for entry in column:
        for i in range(0, len(column.index)):
            if "Adventure" in entry:
                movies_df["Adventure"].loc[i]  = 1
            else: 
                movies_df["Adventure"].loc[i] = 0
            if "Action" in entry:
                movies_df["Action"].loc[i] = 1
            else: 
                movies_df["Action"].loc[i] = 0
            if "Animation" in entry:
                movies_df["Animation"].loc[i] = 1
            else: 
                movies_df["Animation"].loc[i] = 0
            if "Children" in entry:
                movies_df["Children"].loc[i] = 1
            else: 
                movies_df["Children"].loc[i] = 0
            if "Comedy" in entry:
                movies_df["Comedy"].loc[i] = 1
            else: 
                movies_df["Comedy"].loc[i] = 0
            if "Crime" in entry:
                movies_df["Crime"].loc[i] = 1
            else: 
                movies_df["Crime"].loc[i] = 0
            if "Documentary" in entry:
                movies_df["Documentary"].loc[i] = 1
            else: 
                movies_df["Documentary"].loc[i] = 0
            if "Drama" in entry:
                movies_df["Drama"].loc[i] = 1
            else: 
                movies_df["Drama"].loc[i] = 0
            if "Fantasy" in entry:
                movies_df["Fantasy"].loc[i] = 1
            else: 
                movies_df["Fantasy"].loc[i] = 0
            if "Film-Noir" in entry:
                movies_df["Film-Noir"].loc[i] = 1
            else: 
                movies_df["Film-Noir"].loc[i] = 0
            if "Horror" in entry:
                movies_df["Horror"].loc[i] = 1
            else: 
                movies_df["Horror"].loc[i] = 0
            if "IMAX" in entry:
                movies_df["IMAX"].loc[i] = 1
            else: 
                movies_df["IMAX"].loc[i] = 0
            if "Musical" in entry:
                movies_df["Musical"].loc[i] = 1
            else: 
                movies_df["Musical"].loc[i] = 0
            if "Mystery" in entry:
                movies_df["Mystery"].loc[i] = 1
            else: 
                movies_df["Mystery"].loc[i] = 0
            if "Romance" in entry:
                movies_df["Romance"].loc[i] = 1
            else: 
                movies_df["Romance"].loc[i] = 0
            if "Sci-Fi" in entry:
                movies_df["Sci-Fi"].loc[i] = 1
            else: 
                movies_df["Sci-Fi"].loc[i] = 0
            if "Thriller" in entry:
                movies_df["Thriller"].loc[i] = 1
            else: 
                movies_df["Thriller"].loc[i] = 0
            if "War" in entry:
                movies_df["War"].loc[i] = 1
            else: 
                movies_df["War"].loc[i] = 0
            if "Western" in entry:
                movies_df["Western"].loc[i] = 1
            else: 
                movies_df["Western"].loc[i] = 0
            if "None" in entry:
                movies_df["No Listed Genre"].loc[i] = 1
            else: 
                movies_df["No Listed Genre"].loc[i] = 0
            if "(no genres listed)" in entry:
                movies_df["No Listed Genre"].loc[i] = 1
            else:
                movies_df["No Listed Genre"].loc[i] = 0
    return movies_df.head(10)

genre_split(movies_df["genres"])

# Reorder Columns In Final DataFrame

In [21]:
movies_df = movies_df.loc[:, ['Title', 'Year', 'Adventure', 'Action', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'No Listed Genre']]

SyntaxError: invalid syntax (Temp/ipykernel_15144/1330765265.py, line 1)

# Drop NaN Values From Other DataFrames

In [ ]:
credit_df.dropna()
link_df.dropna()
meta_df.dropna()
ratings_df.dropna()

# Write All DataFrames To CSV Files

In [ ]:
movies_df.to_csv("Data/movies_clean.csv")
credit_df.to_csv("Data/credits_clean.csv")
link_df.to_csv("Data/links_clean.csv")
meta_df.to_csv("Data/metadata_clean.csv")
ratings_df.to_csv("Data/ratings_clean.csv")
